In [ ]:
!pip install --upgrade git+https://github.com/oKatanaaa/GCT

In [ ]:
!pip install --upgrade git+https://github.com/MakiResearchTeam/MakiFlow.git

In [ ]:
%tensorflow_version 1.x

In [ ]:
import tensorflow as tf

In [ ]:
from gct.tools import load_boston_housing
import matplotlib.pyplot as plt
from makiflow.models.regressor import Regressor, MseTrainer
from makiflow.generators.classification import cycle_generator
import tensorflow as tf
from makiflow.layers import *
import numpy as np
%matplotlib inline

In [ ]:
(Xtrain, Ytrain), (Xtest, Ytest), (X_mean, X_std), (Y_mean, Y_std) = load_boston_housing()

In [ ]:
Xtrain.shape

In [ ]:
BATCH_SIZE = 64

In [ ]:
def block(x, id, in_d, out_d):
    SX = x
    FX = x
    
    FX = BatchNormLayer(D=in_d, name=f'bn_main_{id}_1')(FX)
    FX = ActivationLayer(name=f'act_1_{id}')(FX)
    FX = DenseLayer(in_d=in_d, out_d=out_d, name=f'dense_main_{id}_1', activation=None)(FX)
    
    FX = BatchNormLayer(D=out_d, name=f'bn_main_{id}_2')(FX)
    FX = ActivationLayer(name=f'act_2_{id}')(FX)
    FX = DenseLayer(in_d=out_d, out_d=out_d, name=f'dense_main_{id}_2', activation=None)(FX)

    if in_d != out_d:
        SX = BatchNormLayer(D=in_d, name=f'bn_skip_{id}_1')(SX)
        SX = DenseLayer(in_d=in_d, out_d=out_d, name=f'conv_skip_{id}', activation=None)(SX)
    
    x_sum = SumLayer(name=f'sum_{id}')([FX, SX])
    return x_sum

In [ ]:
# Create a model

In [ ]:
inp = InputLayer(input_shape=[BATCH_SIZE, 13], name='input')
x = block(inp, id=1, in_d=13, out_d=128)
x = block(x, id=2, in_d=128, out_d=128)
x = block(x, id=5, in_d=128, out_d=128)

x = DenseLayer(in_d=128, out_d=1, activation=None, name='classification_head')(x)

In [ ]:
model = Regressor(in_x=inp, out_x=x)
sess = tf.Session()
model.set_session(sess)

In [ ]:
# Setup training

In [ ]:
trainer = MseTrainer(model=model, train_inputs=[InputLayer(input_shape=[BATCH_SIZE, 13], name='input')])

In [ ]:
trainer.compile()

In [ ]:
opt = tf.train.AdamOptimizer()

In [ ]:
gen = cycle_generator(Xtrain, Ytrain, BATCH_SIZE)

In [ ]:
info1 = trainer.fit_generator(gen, optimizer=opt, epochs=2, iter=100, print_period=10)

In [ ]:
plt.plot(info1[MseTrainer.TRAINING_LOSS])

In [ ]:
model.evaluate(Xtest, Ytest)